<a href="https://colab.research.google.com/github/tmehtiyev2019/llm-api-tm/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Dependencies
!pip install openai
!pip install langchain

In [5]:
#@title Import necessary libraries
from typing import Optional
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_openai_fn_runnable,
    create_structured_output_chain,
    create_structured_output_runnable,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from typing import Sequence
from langchain.pydantic_v1 import BaseModel, Field
import pandas as pd
from google.colab import drive
from langchain.llms import OpenAI
import openai

In [6]:
drive.mount('/content/drive')
api_key_path = '/content/drive/My Drive/api_key.txt'  # Update this path to the location of your API key file
with open(api_key_path, 'r') as file:
    openai_api_key = file.read().strip()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
def process_user_input(input_text, model):
    llm = OpenAI(
    api_key=openai_api_key,
    model_name=model,  # or another model you prefer that is supported
    temperature=0.5,
    max_tokens=64)
    try:
        # Generating a response from the LLM using LangChain
        response = llm.generate(
            prompts=[input_text],
            max_tokens=150
        )

        # Extracting the generated text from the nested structure
        # Access the first 'Generation' object in the first list of the 'generations' list
        # Then access the 'text' attribute of this 'Generation' object
        generated_text = response.generations[0][0].text
        return generated_text
    except Exception as e:
        # Catching and displaying any errors encountered
        print(f"Error encountered: {e}")
        return None


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class algorithm for extracting product information in structured formats."),
    ("human", "Use the given format to extract product information from the following input: {input}"),
    ("human", "Tip: Make sure to answer in the correct format"),
])
json_schema = {
    "title": "APIRequest",
    "description": "Schema for creating a new product in the catalog.",
    "type": "object",
    "properties": {
        "method": {"type": "string", "enum": ["POST"]},
        "endpoint": {"type": "string", "const": "/products"},
        "description": {"type": "string"},
        "requestBody": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "price": {"type": "integer"},
                "quantity": {"type": "integer"}
            },
            "required": ["name", "price", "quantity"]
        }
    },
    "required": ["method", "endpoint", "description", "requestBody"]
}


def api_structure_output(model, llm_response, json_schema, prompt):
  llm = ChatOpenAI(api_key=openai_api_key, model=model, temperature=0)
  runnable = create_structured_output_runnable(json_schema, llm, prompt)
  # Invoke the runnable to get the structured API request
  api_request = runnable.invoke({"input": llm_response})
  return api_request
  # print(api_request)


# # Initialize the runnable
# runnable = create_runnable()

# Example user input
user_input = "Generate a sentence as if you want to add details of products to your product catalog including its name, price, and quantity in stock. But these sentences should not be easy to understand particularly for LLM models. The formats should vary not montonic"

# DataFrame to store the results
results_df = pd.DataFrame(columns=["LLM Response", "API Structure Result from GPT-3.5", "API Structure Result from GPT-4"])


print(process_user_input(user_input, "text-davinci-003"))

# Generate responses and structure them
for _ in range(2):
    api_input = process_user_input(user_input, "text-davinci-003")

    api_request_3_5 = api_structure_output("gpt-4", api_input, json_schema, prompt)

    api_request_4 = api_structure_output("gpt-3.5-turbo-1106", api_input, json_schema, prompt)

    results_df = results_df.append({
        "LLM Response":  str(api_input),
        "API Structure Result from GPT-3.5": api_request_3_5,
        "API Structure Result from GPT-4": api_request_4
    }, ignore_index=True)

print(results_df)




"We have the latest iteration of the product, with a price tag of $50 and an inventory of 25 units."


<ipython-input-7-fb86b4a5e3c6>:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


                                        LLM Response  \
0  \n\n"This item, with a price tag of $XX and a ...   
1  .\n\n"This item is a must-have for any collect...   

                   API Structure Result from GPT-3.5  \
0  {'method': 'POST', 'endpoint': 'http://api.exa...   
1  {'method': 'POST', 'endpoint': 'http://example...   

                     API Structure Result from GPT-4  
0  {'method': 'POST', 'endpoint': '/products', 'd...  
1  {'method': 'POST', 'endpoint': '/products', 'd...  


<ipython-input-7-fb86b4a5e3c6>:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [8]:
results_df

,LLM Response,API Structure Result from GPT-3.5,API Structure Result from GPT-4
0,"\n\n""This item, with a price tag of $XX and a ...","{'method': 'POST', 'endpoint': 'http://api.exa...","{'method': 'POST', 'endpoint': '/products', 'd..."
1,".\n\n""This item is a must-have for any collect...","{'method': 'POST', 'endpoint': 'http://example...","{'method': 'POST', 'endpoint': '/products', 'd..."


In [57]:
results_df.to_excel('output.xlsx', index=False)